In [17]:
import numpy as np
import tensorflow as tf
from string import punctuation
from collections import Counter
import collections

In [18]:
with open('rev_3.txt', 'r') as f:
    reviews = f.read()
with open('lab.txt', 'r') as f:
    labels_org = f.read()

In [19]:
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')
all_text = ' '.join(reviews)
words = all_text.split()

In [20]:
"""
counts = Counter(words).most_common(2000 - 1)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
int_to_vocab=dict(zip(vocab_to_int.values(),vocab_to_int.keys()))
int_to_vocab[0]='UNK'
reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])
"""

"\ncounts = Counter(words).most_common(2000 - 1)\nvocab = sorted(counts, key=counts.get, reverse=True)\nvocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}\nint_to_vocab=dict(zip(vocab_to_int.values(),vocab_to_int.keys()))\nint_to_vocab[0]='UNK'\nreviews_ints = []\nfor each in reviews:\n    reviews_ints.append([vocab_to_int[word] for word in each.split()])\n"

In [21]:
vocabulary_size = 400
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    i=0
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, vocab_to_int, int_to_vocab = build_dataset(words,
                                                            vocabulary_size)
reviews_ints=[]
for each in reviews:
    reviews_ints.append([vocab_to_int.get(word,0) for word in each.split()])
reviews_ints[0:10]

[[312, 0, 5, 11, 391, 12, 20, 46, 392, 140, 65, 0, 4, 393, 24],
 [5, 0, 0, 313, 82, 211, 0],
 [5, 0, 394, 0, 92, 0, 0, 4, 314, 6],
 [188, 212, 213],
 [5, 246, 189, 395, 5, 246, 189],
 [0, 58, 3, 5, 0, 11, 3, 0, 0, 0, 0, 315, 0, 0, 13, 0, 11, 0, 0],
 [5, 190, 0, 3, 0, 30, 0, 0, 22, 0, 214],
 [5, 190, 4, 0, 3, 0, 214, 22, 3, 0, 9, 0],
 [0, 0, 113, 55, 5, 0, 0, 93],
 [5, 53, 0, 0, 164, 8, 165, 0, 0, 0]]

In [22]:
import pickle
pickle_out = open("vocab_to_int.pickle","wb")
pickle.dump(vocab_to_int, pickle_out)
pickle_out.close()

In [6]:
#labels = np.array([1 if l == "positive" else 0 for l in labels_org.split()])
labels = np.array([int(_)  for _ in labels_org.split()]).astype(np.float32)
print(len(labels))
print(labels[:10])

1000
[ 1.  1.  1.  1.  1. -1.  1.  1.  1.  1.]


In [7]:
A=[len(x) for x in reviews_ints]
review_lens = Counter(A)
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 2
Maximum review length: 30


In [8]:
reviews_ints = [r[0:30] for r in reviews_ints if len(r) > 0]
print(len(reviews_ints),reviews_ints[0])
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

1000 [312, 0, 5, 11, 391, 12, 20, 46, 392, 140, 65, 0, 4, 393, 24]
Zero-length reviews: 0
Maximum review length: 30


In [9]:
seq_len = 30
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
features[0,:]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 312,   0,   5,  11, 391,  12,  20,  46, 392, 140,  65,
         0,   4, 393,  24])

In [23]:
T=[int_to_vocab.get(features[0,i],'UNK') for i in range(30)]
print(T)
del T

['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'Which', 'UNK', 'BMW', 'is', 'right', 'for', 'you', 'We', 'test', 'them', 'all', 'UNK', 'to', 'find', 'out']


In [24]:
from sklearn.model_selection import train_test_split

train_x,XX,train_y,YY=train_test_split(features,labels,test_size=0.2)
val_x, test_x,val_y, test_y=train_test_split(XX,YY,test_size=0.5)

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(800, 30) 
Validation set: 	(100, 30) 
Test set: 		(100, 30)
label set: 		(800,) 
Validation label set: 	(100,) 
Test label set: 		(100,)


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   1,   0, 320],
       [  0,   0,   0, ...,   8, 258, 320],
       ..., 
       [  0,   0,   0, ...,   6,   0, 320],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0, 199, ...,   0,   0,   0]])

In [12]:
lstm_size = 25
lstm_layers = 1
batch_size = 50
learning_rate = 0.001
embed_size = 30 
n_words = len(vocab_to_int) #+ 1 # Add 1 for 0 added to vocab
print(n_words)

400


In [13]:
# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, 30], name="inputs")
    labels_ = tf.placeholder(tf.float32, [None,], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [14]:

def lstm_cell():
    #lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    lstm=tf.contrib.rnn.BasicRNNCell(num_units=lstm_size,reuse=tf.get_variable_scope().reuse)
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
with tf.name_scope("RNN_layers"):
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32) 
    
with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)  
    
with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=None)
    
with tf.name_scope('cost'):
    #cost = tf.losses.mean_squared_error(labels_, predictions)
    #cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=predictions))
    cost=tf.reduce_mean(tf.squared_difference(labels_,predictions))
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope('validation'):
    #correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    #accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    #correct = tf.nn.in_top_k(predictions,labels_ , 1)
    #accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    total_error = tf.reduce_sum(tf.square(labels_ - tf.reduce_mean(labels_)))
    unexplained_error = tf.reduce_sum(tf.square(labels_ - predictions))
    accuracy = 1- tf.div(total_error, unexplained_error)
    
tf.summary.histogram('predictions', predictions)
tf.summary.scalar('cost', cost)
merged = tf.summary.merge_all()

In [15]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [16]:
epochs = 10
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, "checkpoints/sentiment_2.ckpt")
    train_writer = tf.summary.FileWriter('train_I_II', sess.graph)
    test_writer = tf.summary.FileWriter('test_I_II', sess.graph)
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: np.array(y[:, None]).reshape(-1),
                    keep_prob: 1,
                    initial_state: state}
            summary, loss, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)
#            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)

            train_writer.add_summary(summary, iteration)
        
            if iteration%10==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%10==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: np.array(y[:, None]).reshape(-1),
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
            test_writer.add_summary(summary, iteration)
            saver.save(sess, "checkpoints/sentiment.ckpt")
    saver.save(sess, "checkpoints/sentiment_final_RNN.ckpt")

Epoch: 0/10 Iteration: 10 Train loss: 0.879
Val acc: 0.986
Epoch: 1/10 Iteration: 20 Train loss: 0.765
Val acc: 0.986
Epoch: 1/10 Iteration: 30 Train loss: 0.586
Val acc: 0.984
Epoch: 2/10 Iteration: 40 Train loss: 0.701
Val acc: 0.983
Epoch: 3/10 Iteration: 50 Train loss: 0.693
Val acc: 0.982
Epoch: 3/10 Iteration: 60 Train loss: 0.674
Val acc: 0.982
Epoch: 4/10 Iteration: 70 Train loss: 0.819
Val acc: 0.982
Epoch: 4/10 Iteration: 80 Train loss: 0.768
Val acc: 0.982
Epoch: 5/10 Iteration: 90 Train loss: 0.721
Val acc: 0.981
Epoch: 6/10 Iteration: 100 Train loss: 0.627
Val acc: 0.981
Epoch: 6/10 Iteration: 110 Train loss: 0.484
Val acc: 0.981
Epoch: 7/10 Iteration: 120 Train loss: 0.549
Val acc: 0.981
Epoch: 8/10 Iteration: 130 Train loss: 0.654
Val acc: 0.981
Epoch: 8/10 Iteration: 140 Train loss: 0.628
Val acc: 0.981
Epoch: 9/10 Iteration: 150 Train loss: 0.793
Val acc: 0.981
Epoch: 9/10 Iteration: 160 Train loss: 0.741
Val acc: 0.981


In [209]:
test_acc = []
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/sentiment.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: np.array(y[:, None]).reshape(-1),
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/sentiment.ckpt
Test accuracy: 0.981
